In [1]:
# Load documents
# split texts
# convert to embeddings
# store it in vector store
# get retriever from vector store
# retrieve relevant documents using retriever
# create a template (map_template) so that fetch info related to query from each doc and append the output of each doc queried in an array
# create a template (fusion_template) so that refined answer should be displayedfrom the commbined output of the stored array by removing duplicates, displaying contradiction if any and displaying factual information
# loop each docs retrieved from retriever and invoke llm prompt with map_template for each doc and store each output in an array
# invoke llm with fusion_template and display the answer.


In [ ]:
# Load documents
from langchain_community.document_loaders import PyMuPDFLoader
import os
all_docs = []
for file in os.listdir("./"):
    if(file.endswith(".pdf")):
        file_path = os.path.join("./", file)
        print(file_path)
        loader = PyMuPDFLoader(file_path=file_path)
        all_docs.extend(loader.load())
        


# split texts
from langchain_text_splitters import CharacterTextSplitter
text_splitter = CharacterTextSplitter.from_tiktoken_encoder(chunk_size=1000, chunk_overlap=50)
splits = text_splitter.split_documents(all_docs)
splits


./english-10.pdf
./ADD-953.pdf
./playwright-mcp-server.pdf


[Document(metadata={'producer': 'pypdf', 'creator': '', 'creationdate': '', 'source': './english-10.pdf', 'file_path': './english-10.pdf', 'total_pages': 224, 'format': 'PDF 1.7', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'moddate': '', 'trapped': '', 'modDate': '', 'creationDate': '', 'page': 0}, page_content='GOVERNMENT OF TAMIL NADU\nENGLISH\nA publication under Free Textbook Programme of Government of Tamil Nadu\nDepartment of School Education\nSTANDARD TEN\nUntouchability is Inhuman and a Crime\n10th English_Preface.indd   1\n28-11-2022   18:18:13\nwww.tntextbooks.in'),
 Document(metadata={'producer': 'pypdf', 'creator': '', 'creationdate': '', 'source': './english-10.pdf', 'file_path': './english-10.pdf', 'total_pages': 224, 'format': 'PDF 1.7', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'moddate': '', 'trapped': '', 'modDate': '', 'creationDate': '', 'page': 1}, page_content='Government of Tamil Nadu\nFirst Edition\t\n-\t2019\nRevised Edition\t -\t

In [12]:
# convert to embeddings
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_core.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, SystemMessagePromptTemplate
embedder = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

# store it in vector store
from langchain_community.vectorstores import FAISS
vector_store = FAISS.from_documents(splits, embedder)

# get retriever from vector store
retriever = vector_store.as_retriever(k=5) # Get top 5 relevant docs

# Create llm
from langchain_google_genai import GoogleGenerativeAI
import os
from dotenv import load_dotenv
load_dotenv()
os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY")
llm = GoogleGenerativeAI(model="gemini-2.5-flash")

# Create parser
from langchain_core.output_parsers import StrOutputParser
parser = StrOutputParser()


# create a template (map_template) so that fetch info related to query from each doc and append the output of each doc queried in an array
map_template = ChatPromptTemplate.from_messages(
    [
        SystemMessagePromptTemplate.from_template("Extract relevent info that matches the query {question} from doc {document}"),
        HumanMessagePromptTemplate.from_template("{question}")
    ]
)

# create a template (fusion_template) so that refined answer should be displayedfrom the commbined output of the stored array by removing duplicates, displaying contradiction if any and displaying factual information
fusion_template = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template("Combine the following pieces of information into a single, coherent answer. Remove duplicates, mentione contradictions if any under 'Contradiction:' , and present factual information."),
    HumanMessagePromptTemplate.from_template("Information pieces: {summaries}")
])



/var/folders/2w/cqz3z_yn04n_r99f8b5w1frc0000gp/T/ipykernel_6141/1813276578.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embedder = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")


In [14]:
mapped_answers = []
def fusion_rag(question):
    # retrieve relevant documents using retriever
    relevant_docs = retriever._get_relevant_documents(question)
    for doc in relevant_docs:
        mapped = llm.invoke(map_template.format_messages(question= question, document= doc)).content
        mapped_answers.append(mapped)

    return llm.invoke(
        llm.invoke(fusion_template.format_messages(summaries= '\n\n---\n\n'.join(mapped_answers)))
    )
result = fusion_rag("Why was the TVE authorization model chosen over MVPD subscriptions?")

# loop each docs retrieved from retriever and invoke llm prompt with map_template for each doc and store each output in an array
# invoke llm with fusion_template and display the answer.

TypeError: VectorStoreRetriever._get_relevant_documents() missing 1 required keyword-only argument: 'run_manager'